数据集基类的标准化流程：
load metainfo-> join path(应该是data_root和image_prefix、ann_file的拼接)->build pipeline->
load data_list->...

In [2]:
from mmengine.dataset import BaseDataset
import os.path as osp

class ToyDataset(BaseDataset):
    def parse_data_info(self,raw_data_info):
        #raw_data_info代表list[dict]中的一个字典
        data_info=raw_data_info
        image_prefix=self.data_prefix.get('img_path',None)
        if image_prefix is not None:
            data_info['img_path']=osp.join(image_prefix,data_info['img_path'])
        return data_info
#注意img_path字段，既是图像前缀的字段名，也是图像文件的字段名（不如mmcv的filename）

In [3]:

class LoadImage:

    def __call__(self, results):
        results['img'] = cv2.imread(results['img_path'])
        return results

class ParseImage:

    def __call__(self, results):
        results['img_shape'] = results['img'].shape
        return results

pipeline = [
    LoadImage(),
    ParseImage(),
]

toy_dataset = ToyDataset(
    data_root='data/',
    data_prefix=dict(img_path='train/'),
    ann_file='annotations/train.json',
    pipeline=pipeline)

FileNotFoundError: [Errno 2] No such file or directory: 'data/annotations/train.json'

一个原始数据可能只包含一个训练样本，也可能包含多个训练样本，比如视频帧

parse_data_info方法解析数据，他的入参raw_data_info是调用了load_data_info函数后的返回值，是
list[dict],所以对于不满足规范的数据集，重载该方法即可。

懒加载，配置文件的lazy_init=True，只会执行前三个步骤，即load meatinfo,join path,build pipeline
后续要是用数据集的信息，需要手动执行dataset.full_init()

数据集基类包装器


In [ ]:
#连接多个数据集
from mmengine.dataset import ConcatDataset

dataset1,dataset2=ToyDataset(),ToyDataset()
my_dataset=ConcatDataset(datasets=[dataset1,dataset2])

#不难推导配置文件写法为
my_dataset=dict(
    type='ConcatDataset',
    datasets=[dict(type='ToyDataset'),dict(type='ToyDataset')]
)

In [ ]:
#重复采样数据集
from mmengine.dataset import RepeatDataset
my_dataset=RepeatDataset(ToyDataset(),times=1000)

In [ ]:
#类别平衡数据集
#注意：该数据集包装器需要被包装的数据集支持get_cat_ids方法，也就是根据某idx获取其训练样本的类别
from mmengine.dataset import ClassBalancedDataset,BaseDataset

class ToyDataset(BaseDataset):
    def parse_data_info(self,raw_data_info):
        #raw_data_info代表list[dict]中的一个字典
        data_info=raw_data_info
        image_prefix=self.data_prefix.get('img_path',None)
        if image_prefix is not None:
            data_info['img_path']=osp.join(image_prefix,data_info['img_path'])
        return data_info
    
    def get_cat_ids(self,idx):
        data_info=self.get_data_info(idx)
        return [int(data_info['img_label'])]
my_dataset=ClassBalancedDataset(dataset=ToyDataset(),oversample_thr=0.3)

In [ ]:
from mmengine.dataset import BaseDataset
from mmengine.registry import DATASETS
import copy

#如果需要自定义数据集包装器，需要实现以下的一些部分
DATASETS.register_module()
class ExampleDatasetWrapper():

    def __init__(self,dataset,lazy_init=False):
        #构建原始数据集
        if isinstance(dataset,dict):
            self.dataset=DATASETS.build(dataset)
        elif isinstance(dataset,BaseDataset):
            self.dataset=dataset
        else:
            raise TypeError('dataset must be a dict or BaseDataset')
        
        self._metainfo=self.dataset.metainfo
        """
        1.包装器的超参数
        """

        self._full_initalized=False

        if not lazy_init:
            self.init_full()
    
    def full_init(self):
        if self._full_initalized:
            return
        
        self.full_init()
        """"2.实现包装数据集"""
        self._full_initalized=True

    @force_full_init
    def _get_ori_dataset_idx(self,idx):
        """3.实现将索引idx映射到原始数据集的索引ori_idx"""
        ori_idx=2*idx
        return ori_idx
    #以下是提供与原始dataset一致的对外接口

    @force_full_init
    def get_data_info(self,idx):
        sample_idx=self._get_ori_dataset_idx(idx)
        return self.dataset.get_data_info(sample_idx)
    
    def __getitem__(self,idx):
        if not self._fully_initialized:
            warnings.warn('Please call `full_init` method manually to '
                          'accelerate the speed.')
            self.full_init()

        sample_idx = self._get_ori_dataset_idx(idx)
        return self.dataset[sample_idx]
    
    @force_full_init
    def __len__(self):
        """4.实现获取包装数据集的数据长度"""
        len_wrapper=len(self.dataset)
        return len_wrapper
    
    @property
    def metainfo(self):
        return copy.deepcopy(self._metainfo)
